In [60]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import cmath
import sys
import IPython.display as ipd

from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from scipy import signal

In [14]:
"""
Open the training data.
"""
training_data_filepath = 'F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/output/audio_training_data_dict.pickle'

with open(training_data_filepath, 'rb') as f:
    data_dict = pickle.load(f)
    
for key in data_dict:
    print(key, 'is of shape', data_dict.get(key).shape)

train_input is of shape (1048, 88200)
val_input is of shape (225, 88200)
test_input is of shape (224, 88200)
train_targets is of shape (1048, 88200)
val_targets is of shape (225, 88200)
test_targets is of shape (224, 88200)
train_noise is of shape (1048, 88200)
val_noise is of shape (225, 88200)
test_noise is of shape (224, 88200)


In [8]:


# TODO: the paper refers to y(t) as the "additive mixture", so maybe it shouldn't be the average
y = data_dict.get('train_input')[:,0:256]
x = data_dict.get('train_targets')[:,0:256]
n = data_dict.get('train_noise')[:,0:256]
Sy = compute_S(y)

In [15]:
y = data_dict.get('train_input')[0]
x = data_dict.get('train_targets')[0]
n = data_dict.get('train_noise')[0]
# Sy = compute_S(y)
# Sx = compute_S(x)
# Sn = compute_S(n)

In [17]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.stft.html
f, t, Zxx = signal.stft(x, 22050, nperseg=256)
print('x.shape:', x.shape)
print('Zxx.shape:', Zxx.shape)
print(x[0])

x.shape: (88200,)
Zxx.shape: (129, 691)
0.001373291


In [46]:
f = 22050

def compute_S(arr, f):
    T = len(arr)
    window_mults = np.hamming(T)
    S_list = []
    
    for tau in range(0, T):
        summation = complex(0, 0)

        for t in range(0, T):
            summation = summation + complex(arr[t], 0) * \
                                    complex(window_mults[t-tau], 0) * \
                                    cmath.exp(-2j*math.pi*f*t/T)
            
        S_list.append(summation)
        
        
        print("%2.2f%% done" % (float(tau+1)/float(T)*100.0), end="\r")
        
        
    return np.array(S_list)



![title](equation.png)


In [50]:
Sy = compute_S(y[0:len(y)//4], f)

In [52]:
print(y[0:len(y)//4].shape, Sy.shape)

(22050,) (22050,)


In [53]:
Sx = compute_S(x[0:len(x)//4], f)
print(x[0:len(x)//4].shape, Sx.shape)

(22050,) (22050,)


In [54]:
Sn = compute_S(n[0:len(n)//4], f)
print(n[0:len(n)//4].shape, Sn.shape)

(22050,) (22050,)


In [55]:
Px = np.square(Sx)
Pn = np.square(Sn)
R = Px / sum([Px, Pn])

In [56]:
Sx_hat = R * Sy

In [57]:
def S_to_wav(arr, f):
    T = len(arr)
    window_mults = np.hamming(T)
    audio_list = []
    
    for tau in range(0, T):
        summation = complex(0, 0)

        # it says f here, but is that right? in the paper
        for t in range(0, T):
            summation = summation + complex(arr[tau], 0) * \
                                    complex(window_mults[t-tau], 0) * \
                                    cmath.exp(2j*math.pi*f*t/T)
            
            
        audio_list.append(summation)
        
        
        print("%2.2f%% done" % (float(tau+1)/float(T)*100.0), end="\r")
        
        
    return np.array(audio_list)/T

![title](equation.png)


In [58]:
x_hat = S_to_wav(Sx_hat, 22050)

In [61]:
ipd.Audio(x_hat, rate=22050)

C:\Users\ZaknafeinII\anaconda3\lib\site-packages\IPython\lib\display.py:159: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(data, dtype=float)


In [62]:
ipd.Audio(x[0:len(x)//4], rate=22050)